In [9]:
import cassandra

In [10]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [11]:
# Creating the keyspace (~ database)
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS data_modelling
        WITH REPLICATION =
            {
                'class' : 'SimpleStrategy',
                'replication_factor' : 1
            }
    """)
except Exception as e:
    print(e)

In [12]:
# Connect to the keyspace
try:
    session.set_keyspace('data_modelling')
except Exception as e:
    print(e)

<h2>Questions for our data</h2>
<h4>1. Give me every album in the music library that was released in an year</h4>
SELECT * FROM MUSIC_LIBRARY WHERE YEAR = 1970
<h4>2. Give me every album in my artist library that was created from a specific artist</h4>
SELECT * FROM ARTIST_LIBRARY WHERE ARTIST_NAME = 'THE BEATLES'

In [13]:
query = 'CREATE TABLE IF NOT EXISTS music_library '
query += '(YEAR INT, ARTIST_NAME TEXT, ALBUM_NAME TEXT, PRIMARY KEY (YEAR, ARTIST_NAME))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
query = 'INSERT INTO music_library (YEAR, ARTIST_NAME, ALBUM_NAME)'
query += ' VALUES (%s, %s, %s)'
try:
    session.execute(query, (1970, 'The Beatles', 'Let It Be'))
    session.execute(query, (1965, 'The Beatles', 'I Will Win'))
except Exception as e:
    print(e)

In [22]:
query = 'CREATE TABLE IF NOT EXISTS artist_library '
query += '(YEAR INT, ARTIST_NAME TEXT, ALBUM_NAME TEXT, PRIMARY KEY (ARTIST_NAME, YEAR))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
query = 'INSERT INTO artist_library (YEAR, ARTIST_NAME, ALBUM_NAME)'
query += ' VALUES (%s, %s, %s)'
try:
    session.execute(query, (1890, 'The Beatles', 'Let It Be'))
    session.execute(query, (1976, 'The Beatles', 'I Will Win'))
except Exception as e:
    print(e)

In [17]:
# Create table
try:
    rows = session.execute('SELECT * FROM music_library WHERE YEAR = 1970')
except Exception as e:
    print(e)
for row in rows:
    print(row.artist_name, row.year, row.album_name)

The Beatles 1970 Let It Be


In [24]:
# Create table
try:
    rows = session.execute("SELECT * FROM artist_library WHERE artist_name = 'The Beatles'")
except Exception as e:
    print(e)
for row in rows:
    print(row.artist_name, row.year, row.album_name)

The Beatles 1890 Let It Be
The Beatles 1976 I Will Win


In [19]:
try:
    rows = session.execute('DROP TABLE music_library;')
except Exception as e:
    print(e)

In [25]:
try:
    rows = session.execute('DROP TABLE artist_library;')
except Exception as e:
    print(e)

In [26]:
session.shutdown()
cluster.shutdown()